In [1]:
%pylab inline
import pandas as pd
import os
from artm_experiments import OptimizationTopicsFilter
from optimize_methods import cosine_metric, jac_cosine_metric
from artm import *
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


/opt/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def read_phi(phi_name, topics_name):
    phi = pd.read_pickle(os.path.join('phi_ethnic', phi_name))
    topics = dict()
    to_class = {'++': 0, '+-': 1, '-+': 2, '--': 3}
    with open(os.path.join('ethnic_top_tokens', topics_name)) as f:
        for i in xrange(400):
            line = f.readline().split()
            topics[line[1][:-1]] = to_class[line[0]]
    return phi, topics

In [3]:
phi_hull, topics_hull = read_phi('4_phi_smooth_decor', 'smooth_decor.txt')
phi_train, topics_train = read_phi('5_phi_full', 'full.txt')
phi_test, topics_test = read_phi('2_phi_lda', 'lda.txt')

In [4]:
hull_euc = OptimizationTopicsFilter(eps=1e-5)
hull_cos = OptimizationTopicsFilter(eps=1e-5, metric=cosine_metric, jac_metric=jac_cosine_metric)

topics_classes = {0: [], 1: [], 2: [], 3: []}

for topic in phi_hull.columns:
    topics_classes[topics_hull[topic]].append(topic)

In [5]:
from sklearn.metrics.pairwise import cosine_distances
from scipy.stats import entropy
import time

def cos_metric(u, v):
    return cosine_distances([u], [v])[0, 0]
                                      
def kl_metric(u, v):
    not_zeros = (u > 0.0) & (v > 0.0)
    u_nz = u[not_zeros]
    v_nz = v[not_zeros]
    res = 0.5 * entropy(u_nz, v_nz) + 0.5 * entropy(v_nz, u_nz)
    return min(np.sum(res), 1e3)
                        
def jac_metric(u, v):
    inter = (u > 0.0) & (v > 0.0)
    union = (u > 0.0) | (v > 0.0)
    return 1.0 - 1.0 * np.sum(inter) / np.sum(union)

def find_closest(topic_vector, metric):
    res = []
    for cl in xrange(4):
        min_dist = 1e5
        min_topic = None
        for topic in topics_classes[cl]:
            dist = metric(topic_vector, phi_hull[topic].as_matrix().ravel())
            if dist < min_dist:
                min_dist = dist
                min_topic = topic
                
        res.append(min_dist)
    return res
        

def gen_features(topic_vector):
    features = []
    for cl in xrange(4):
        features.append(hull_euc.get_dist(topic_vector, phi_hull, topics_classes[cl]).fun)
    for cl in xrange(4):
        features.append(hull_cos.get_dist(topic_vector, phi_hull, topics_classes[cl]).fun)
    
    for cut_size in [20, 100, 10 ** 6]:
        for metric in [cos_metric, kl_metric, jac_metric]:
            cutted_vector = topic_vector.copy()
            cutted_vector[np.argsort(cutted_vector)[:-cut_size]] = 0.0
            closest = find_closest(topic_vector, metric)
            features += closest
    #print features
    return np.array(features)

X = []
y = []
for phi, topics in [(phi_train, topics_train), (phi_test, topics_test)]:
    for topic in phi.columns:
        print topic
        X.append(gen_features(phi[topic].as_matrix().ravel()))
        y.append(topics[topic])
    
X = np.array(X)
y = np.array(y)

e_topic_0
e_topic_1
e_topic_2
e_topic_3
e_topic_4
e_topic_5
e_topic_6
e_topic_7
e_topic_8
e_topic_9
e_topic_10
e_topic_11
e_topic_12
e_topic_13
e_topic_14
e_topic_15
e_topic_16
e_topic_17
e_topic_18
e_topic_19
e_topic_20
e_topic_21
e_topic_22
e_topic_23
e_topic_24
e_topic_25
e_topic_26
e_topic_27
e_topic_28
e_topic_29
e_topic_30
e_topic_31
e_topic_32
e_topic_33
e_topic_34
e_topic_35
e_topic_36
e_topic_37
e_topic_38
e_topic_39
e_topic_40
e_topic_41
e_topic_42
e_topic_43
e_topic_44
e_topic_45
e_topic_46
e_topic_47
e_topic_48
e_topic_49
e_topic_50
e_topic_51
e_topic_52
e_topic_53
e_topic_54
e_topic_55
e_topic_56
e_topic_57
e_topic_58
e_topic_59
e_topic_60
e_topic_61
e_topic_62
e_topic_63
e_topic_64
e_topic_65
e_topic_66
e_topic_67
e_topic_68
e_topic_69
e_topic_70
e_topic_71
e_topic_72
e_topic_73
e_topic_74
e_topic_75
e_topic_76
e_topic_77
e_topic_78
e_topic_79
e_topic_80
e_topic_81
e_topic_82
e_topic_83
e_topic_84
e_topic_85
e_topic_86
e_topic_87
e_topic_88
e_topic_89
e_topic_90
e_topic_9

In [8]:
features_names = ['HullEuc{}'.format(cl) for cl in xrange(4)] +\
                 ['HullCos{}'.format(cl) for cl in xrange(4)] +\
                 ['Dist{}{}{}'.format(cut_size, metric, cl) 
                  for cut_size in ['20', '100', 'Inf']
                  for metric in ['Cos', 'Kl', 'Jac'] 
                  for cl in xrange(4) ]
print '\n'.join(features_names)

HullEuc0
HullEuc1
HullEuc2
HullEuc3
HullCos0
HullCos1
HullCos2
HullCos3
Dist20Cos0
Dist20Cos1
Dist20Cos2
Dist20Cos3
Dist20Kl0
Dist20Kl1
Dist20Kl2
Dist20Kl3
Dist20Jac0
Dist20Jac1
Dist20Jac2
Dist20Jac3
Dist100Cos0
Dist100Cos1
Dist100Cos2
Dist100Cos3
Dist100Kl0
Dist100Kl1
Dist100Kl2
Dist100Kl3
Dist100Jac0
Dist100Jac1
Dist100Jac2
Dist100Jac3
DistInfCos0
DistInfCos1
DistInfCos2
DistInfCos3
DistInfKl0
DistInfKl1
DistInfKl2
DistInfKl3
DistInfJac0
DistInfJac1
DistInfJac2
DistInfJac3


In [6]:
X.tofile('x')
#X.tofile('x2')
y.tofile('y')
#y.tofile('y2')

In [9]:
X = np.fromfile('x').reshape((-1, len(features_names)))
y = np.fromfile('y', dtype=int)

In [10]:
X[0]

array([ 0.07943663,  0.07921655,  0.07864999,  0.06417214,  0.99306052,
        0.98923581,  0.99005595,  0.45670187,  0.98139071,  0.98020005,
        0.98538047,  0.45670176,  5.77732301,  6.01411152,  5.14779186,
        2.96902502,  0.73341034,  0.72363516,  0.70723929,  0.70599412,
        0.98139071,  0.98020005,  0.98538047,  0.45670176,  5.77732301,
        6.01411152,  5.14779186,  2.96902502,  0.73341034,  0.72363516,
        0.70723929,  0.70599412,  0.98139071,  0.98020005,  0.98538047,
        0.45670176,  5.77732301,  6.01411152,  5.14779186,  2.96902502,
        0.73341034,  0.72363516,  0.70723929,  0.70599412])

In [11]:
y

array([3, 3, 3, 3, 3, 3, 2, 0, 3, 3, 1, 2, 2, 3, 2, 3, 3, 3, 3, 3, 2, 3, 3,
       3, 3, 3, 3, 1, 3, 0, 3, 3, 3, 0, 3, 3, 3, 1, 3, 3, 2, 1, 3, 1, 3, 3,
       3, 3, 3, 3, 3, 3, 2, 2, 1, 3, 2, 3, 0, 1, 3, 3, 0, 1, 3, 3, 1, 1, 3,
       1, 3, 1, 1, 3, 1, 3, 3, 3, 2, 2, 2, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1,
       1, 2, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 1, 3, 3, 0, 3, 3, 2, 1, 2, 3, 3,
       0, 3, 3, 3, 3, 3, 3, 0, 0, 1, 3, 3, 1, 3, 3, 3, 3, 1, 3, 3, 3, 3, 0,
       2, 0, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 0, 2, 3, 3, 1, 3, 3, 0, 3, 3, 3,
       3, 2, 0, 1, 3, 3, 3, 0, 3, 3, 3, 3, 3, 2, 3, 1, 2, 3, 3, 0, 1, 3, 3,
       2, 0, 0, 3, 3, 0, 3, 3, 1, 3, 3, 3, 3, 3, 3, 1, 1, 3, 1, 1, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 0, 3, 3, 1, 3, 3, 0, 3, 0,
       3, 3, 3, 2, 3, 0, 1, 3, 1, 1, 3, 3, 3, 3, 0, 3, 2, 3, 0, 3, 3, 3, 3,
       3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3,
       3, 3,

In [57]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

#X /= np.max(X, axis=0)
print X[0]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.4)

#model = XGBClassifier(n_estimators=50).fit(X_train, y_train)
model = LogisticRegression(penalty='l1', C=1.0).fit(X_train, y_train)
ans = model.predict_proba(X_test)
for cl in xrange(4):
    binary_test = y_test.copy()
    not_cl = binary_test != cl
    is_cl = binary_test == cl
    binary_test[not_cl] = 0
    binary_test[is_cl] = 1
    
    print roc_auc_score(binary_test, ans[:, cl])

[ 0.11293987  0.11261024  0.11209214  0.096005    0.99389123  0.98981756
  0.99162935  0.46054035  0.98394858  0.98097005  0.98811337  0.46099705
  0.73020088  0.69553384  0.67014844  0.41920739  0.93052007  0.96802121
  0.96717308  0.96682939  0.98394858  0.98097005  0.98811337  0.46099705
  0.73020088  0.69553384  0.67014844  0.41920739  0.93052007  0.96802121
  0.96717308  0.96682939  0.98394858  0.98097005  0.98811337  0.46099705
  0.73020088  0.69553384  0.67014844  0.41920739  0.93052007  0.96802121
  0.96717308  0.96682939]
0.911467410247
0.713378378378
0.766447368421
0.842875157629


In [58]:
def get_importances(model, n_features):
    if not hasattr(model, 'coef_'):
        booster = model.booster()
        fscores = booster.get_fscore()

        importances = np.zeros(n_features)

        for k, v in fscores.iteritems():
            importances[int(k[1:])] = v

        return importances
    else:
        return model.coef_[0]

features_importances = zip(features_names, get_importances(model, X.shape[1]))
features_importances = dict(filter(lambda p: p[1] != 0.0, features_importances))
features_importances = [(name, features_importances[name]) for name in features_names if name in features_importances]

for fi in features_importances:
    print '{},'.format(fi[0])
    
for fi in features_importances:
    print '({},{})'.format(fi[1], fi[0])
    

HullCos1,
HullCos3,
Dist20Cos0,
Dist20Kl0,
Dist20Kl2,
Dist20Jac1,
Dist20Jac2,
Dist100Cos0,
Dist100Kl0,
Dist100Kl2,
Dist100Jac1,
DistInfCos0,
DistInfKl0,
DistInfKl2,
DistInfJac1,
(-2.90706825594,HullCos1)
(2.10014322495,HullCos3)
(-0.906938210154,Dist20Cos0)
(-0.96063409639,Dist20Kl0)
(-0.0344587187901,Dist20Kl2)
(0.451022733161,Dist20Jac1)
(0.0767384529412,Dist20Jac2)
(-0.317719558867,Dist100Cos0)
(-0.821816722347,Dist100Kl0)
(-0.10881609391,Dist100Kl2)
(0.526800976561,Dist100Jac1)
(-0.0303131750142,DistInfCos0)
(-0.626288814545,DistInfKl0)
(-0.160323763677,DistInfKl2)
(0.412879274338,DistInfJac1)
